In [7]:
import numpy as np
import pandas as pd

from ast import literal_eval

df = pd.read_csv('https://raw.githubusercontent.com/Jeremstar/Succes_de_films-IMDb/main/Database/IMDB_2000votes.csv', 

                 converters={"actor": lambda x: x.strip("[]").split(", ")}) 


#on récupère la base de films avec les notes
movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")

movies_rating_beau = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)

movies_rating_beau.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']

movies_rating_beau['Note_moyenne'] = movies_rating_beau['Note_moyenne'].astype(float)

movies_rating_beau['Nombre_de_votes'] = movies_rating_beau['Nombre_de_votes'].astype(float)

movies_rating_épuré = movies_rating_beau[movies_rating_beau.Nombre_de_votes > 1999]



#jointure avec cette table
movies_rating_épuré['url']='/title/'+movies_rating_épuré['ID']+'/'

df_ratings_1 = df.merge(movies_rating_épuré, on='url',how='left')



df_ratings_1 = df_ratings_1.head(100)
df_ratings = df_ratings_1.copy()


df_ratings



#df_ratings['actor'] = df_ratings['actor'].apply(lambda d: d if isinstance(d, list) else [])

#ça c'est la commande magique remplace les Nan mais vu que rien n'est liste elle remplace tout

#plus besoin car la commande d'importation des listes est encore plus magique



#trouver un moyen d'éviter cette redondance

'''

ancienne synthaxe

df_ratings = df_ratings.assign(scores_acteurs=[])

df_ratings = df_ratings.assign(score_actoriel=0)

df_ratings = df_ratings.assign(scores_réalisateurs=[])

df_ratings = df_ratings.assign(score_réalisation=0)

'''

#je crée les nouvelles colonnes mais redndance dans la synthaxe

df_ratings['scores_acteurs'] = ""

df_ratings['score_actoriel'] = ""

df_ratings['scores_réalisateurs'] = ""

df_ratings['score_réalisation'] = ""

df_ratings['scores_acteurs'] = df_ratings['scores_acteurs'].apply(lambda d: d if isinstance(d, list) else [])



#on supprime complétement les colonnes avec des Nan en date : elles sont marginales (j'imagine), on peut se le permettre

index_with_nan = df.index[df_ratings.loc[:,'datePublished'].isnull()]

df_ratings.drop(index_with_nan,0, inplace=True) #trouver un moyen de le mettre dans un autre df ou sinon rip les 700 films sans date

df_ratings.sort_values(by='datePublished')

df_ratings.reset_index(drop = True)





#df_ratings['scores_acteurs'] = df_ratings['actor'].apply(liste_score(actor))



'''

def colone_scoreV1 (metier):  #ne marche pas, probablement car tentative de modifier un df en l'itérant

  if metier == 'actor':

    for i in df_ratings.index :

      for j in df_ratings['actor'][i]: 

        df_ratings['scores_acteurs'][i] = df_ratings['scores_acteurs'][i].append(scoreur(j,i,'actor'))

  elif metier == 'director':

    for i in df_ratings.index :

      for j in df_ratings['director'][i]:

         df_ratings['scores_réalisateurs'][i] = df_ratings['scores_réalisateurs'][i].append(scoreur(j,i,'director'))

  else:

      print("nom de métier invalide") 

'''



def colone_scorev2 (metier):

  if metier == 'actor':

    df_ratings['scores_acteur'] = df_ratings['actor'].apply(lambda d : liste_score(d,df_ratings.index[df_ratings['actor']=='d'],metier)) 

  elif metier == 'director':

    df_ratings['scores_réalisateur'] = df_ratings['director'].apply(lambda d : liste_score(d,df_ratings.index[df_ratings['director']=='d'],metier))

  else:

      print("nom de métier invalide")





def liste_score (liste_personne,index_film,metier):

  liste_score=[]

  for personne in liste_personne :

    liste_score.append(scoreur(personne,index_film,metier))

  return liste_score





def presence(individu, film, metier) :

  if individu in df_ratings[metier][film] :

      indicatrice = True

  else :

      indicatrice = False

  return indicatrice





def scoreur(individu, index_film, metier) :

  somme_notes = 0

  compteur = 0

  if index_film != 0 :

    for film_ant in range(index_film) :

      if presence(individu, film_ant, metier) == True :

        somme_notes = somme_notes + df_ratings['Note_moyenne'][film_ant]

        compteur = compteur + 1

      else:

        print("pas joué dans ce film")

    if compteur!=0 :

      score = somme_notes/compteur

    else:

      score = 5

  else :

    score = 5

  return score



/tmp/ipykernel_1253/3620644562.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_rating_épuré['url']='/title/'+movies_rating_épuré['ID']+'/'


IndexError: boolean index did not match indexed array along dimension 0; dimension is 46784 but corresponding boolean dimension is 100